In [36]:
import numpy as np

import os,sys
module_path = os.path.abspath(os.path.join('epitran'))
if module_path not in sys.path:
  sys.path.append(module_path)

import panphon
import panphon.distance
import epitran
import pickle

from wordgen import *

In [5]:
wg = WordgenLearned(3,"ind-Latn")

Loading Epitran with language code ind-Latn... success!


In [8]:
wg.learn_distribution("slice.txt")

About to learn from slice.txt
For each word, each chunk of 3 sounds will be considered.
Some of the words that could not be processed will be printed below; just check that nothing too bad is happening.
"[[1]]" was not processed.
"|" was not processed.                
"." was not processed.
"," was not processed.                
"," was not processed.
"," was not processed.
"," was not processed.
"." was not processed.
"," was not processed.                
"(" was not processed.
":" was not processed.
")," was not processed.
"." was not processed.
"," was not processed.                
"." was not processed.
"," was not processed.                
";" was not processed.
"," was not processed.
"." was not processed.
"." was not processed.                
"(" was not processed.                
")" was not processed.
"(" was not processed.
")." was not processed.
"|" was not processed.                
"400px" was not processed.
"|" was not processed.
"(" was not processed.
")" was not pro

In [32]:
wg.generate_word()

'd͡ʑəpilmuk'

Now we are interested in generating a "phonology" for a fictional language. This constitutes an equivalence relation on the set of IPA symbols. Then generate an orthography to display things. 

In [41]:
ft=panphon.FeatureTable()
dst=panphon.distance.Distance()

/home/ebrahim/anaconda3/envs/deep-learning/lib/python3.6/site-packages/panphon/distance.py:53: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  dogol_prime = yaml.load(f.read())


In [74]:
dst.weighted_feature_edit_distance('k','g')

7.0